In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import datetime
from tqdm import tqdm 

import warnings 
warnings.filterwarnings('ignore')

In [45]:
#读取数据
df_daily = pd.read_csv(
    'IF8888.CCFX_5m.csv', sep=',', parse_dates=True)
# df_daily = df_daily.iloc[:, 0:3]
df_daily = df_daily[df_daily['date']<='2011-12-09']


df_daily

,date,time,dates,open,close,high,low,volume,money
0,2010-04-16,09:20:00,2010/4/16 9:20,3474.055,3476.633,3513.114,3471.577,2243,2.348769e+09
1,2010-04-16,09:25:00,2010/4/16 9:25,3476.431,3478.663,3483.076,3474.024,1209,1.263114e+09
2,2010-04-16,09:30:00,2010/4/16 9:30,3478.663,3466.988,3479.438,3458.142,1844,1.919536e+09
3,2010-04-16,09:35:00,2010/4/16 9:35,3467.864,3472.736,3478.529,3463.725,1731,1.802158e+09
4,2010-04-16,09:40:00,2010/4/16 9:40,3472.445,3467.961,3475.047,3465.260,1337,1.389845e+09
...,...,...,...,...,...,...,...,...,...
21757,2011-12-09,14:55:00,2011/12/9 14:55,2517.847,2516.331,2518.664,2515.345,2320,1.749707e+09
21758,2011-12-09,15:00:00,2011/12/9 15:00,2516.397,2515.515,2517.071,2514.862,2504,1.887770e+09
21759,2011-12-09,15:05:00,2011/12/9 15:05,2515.541,2515.094,2516.059,2514.768,1612,1.215114e+09
21760,2011-12-09,15:10:00,2011/12/9 15:10,2515.458,2514.306,2515.861,2513.928,1947,1.467425e+09


In [16]:
def VHL(data, sL=2):    
    high = data['high']
    low = data['low']
    date = data['date']
    rank = data['time'].apply(lambda x:pd.Timestamp(x)).groupby(data['date']).rank()
    data["rank"]=rank
    
    vH = high.groupby(date).cummax().shift(sL)
    vL = low.groupby(date).cummin().shift(sL)
    vH[rank<=sL] = high
    vL[rank<=sL] = low
    
    data['vH'] = vH
    data['vL'] = vL
    return data    

In [19]:
def signal(data, default_af=0 , stepsize_af=0.01, max_af=0.1, cL=15):
    close = data['close']
    vH = data['vH']
    vL = data['vL']
    data['signal']=None
    data["stop_price"]=None
    data['stop or not']=False
    direction =None
    stop_price=None
     

    """ 信号计算 """
    for i in range(cL,len(data['close'])):
        
        if direction is None: # 计算开仓方向
            if data["close"][i]>data["vH"][i] : 
                #上穿
                direction =1
                stop_price = data["low"][i]
                data["stop_price"][i+1]=stop_price
                highest_price = data["high"][i]
                af = default_af
                data['signal'][i] = direction
            elif data["close"][i]<data["vL"][i]:
                #下穿
                direction = -1
                stop_price=data["high"][i]
                data["stop_price"][i+1]=stop_price
                lowest_price = data["low"][i]
                af = default_af
                data['signal'][i] = direction

        else:

            if direction == 1 :
                # 已多头入场
                
                #先看当期要不要止损
                if data["low"][i] -stop_price <= 0:
                    #平仓
                    direction = None
                    data['signal'][i] = 0
                    data['stop or not'][i]=True
                
                #再设置止损价格
                if data["high"][i]>highest_price: # 创新高
                    highest_price=data["high"][i]
                    af +=stepsize_af
                    af = af if af <= max_af else max_af
                stop_price = stop_price + (highest_price-stop_price) * af
                data["stop_price"][i+1]=stop_price

                    
            elif direction == -1 :
                # 已空头入场
                
                #先看当期要不要止损
                if stop_price -data["high"][i]  <= 0:
                    #平仓
                    direction = None
                    data['signal'][i] = 0
                    data['stop or not'][i]=True
                
                #再设置止损价格
                if data["low"][i]<lowest_price: # 创新低
                    lowest_price=data["low"][i]
                    af +=stepsize_af
                    af = af if af<=max_af else max_af
                stop_price = stop_price - (stop_price-lowest_price) * af
                data["stop_price"][i+1]=stop_price
                
    return data  

In [23]:
#持仓计算
def position(data, shift_period=1):
    signal = data['signal']
    position = signal.fillna(method='ffill').shift(shift_period).fillna(0)
    data['position'] = position
    return data

In [29]:
def statistic_performance(data, r0=0.03, data_period=1440, 
                          is_consider_open=False,
                          is_consider_stop=False,
                          is_daily_close = False,
                          point = None,
                          comm=None, leverage=None):
    position = data['position']
    close = data['close']
    open_p = data['open']
    
    d_first = data['date'].values[0]
    d_last = data['date'].values[-1]
    d_period = datetime.datetime.strptime(d_last, '%Y-%m-%d') - datetime.datetime.strptime(d_first, '%Y-%m-%d')
    y_period = d_period.days / 365

    # 1. 初始化 
    hold_in_price =  close.shift(1).copy()
    hold_in_price[close.index[0]] = open_p.values[0]
    hold_out_price = close.copy()
              
    # 2. 若考虑持仓信号开盘才能操作
    if is_consider_open:
        # 仓位变动第一期的period_in_price为当日open
        hold_in_price[abs(position - position.shift(1).fillna(0))>0] = open_p
        # 仓位变动前最后一期的period_out_price为下一段的open (除了整个序列最后一天不变)
        hold_out_price[abs(position - position.shift(-1).fillna(0))>0] = open_p.shift(-1)
        hold_out_price[hold_out_price.index[-1]] = close.values[-1]
    
    #3.考虑止损+跳空 比较止损价和开盘价
    if is_consider_stop:
        for i in range(data.index[0],data.index[0]+len(data['close'])):
            if data["position"][i]<0 and data['stop or not'][i] == True:#只有止损平仓stop_or_not是True 15:00:00平仓是False
                hold_out_price[i] = max(data['stop_price'][i],data['open'][i])            
            elif data["position"][i]>0 and data['stop or not'][i] == True:
                hold_out_price[i] = min(data['stop_price'][i],data['open'][i])

    #4.是否收盘前平仓
    if is_daily_close:
        daily_last_idxes = data.drop_duplicates(subset='date',keep='last').index
        #距离每天最后一个时间点的距离 如15:00:00平仓 15:15:00截止  5m一段 hold_out_price.loc[daily_last_idxes.values-3]=close
        hold_out_price.loc[daily_last_idxes.values-3] = close 
    
    #5.考虑指数点
    if point:
        # 仓位变动第一期的period_in_price long加指数点 short减指数点
        hold_in_price[position < position.shift(1).fillna(0)] = hold_in_price - point
        hold_in_price[position > position.shift(1).fillna(0)] = hold_in_price + point       
        # 仓位变动前最后一期的period_out_price long减指数点 short加指数点
        hold_out_price[position < position.shift(-1).fillna(0)] = hold_out_price + point
        hold_out_price[position > position.shift(-1).fillna(0)] = hold_out_price - point
           
    # 6. 计算收益
    hold_r = position * (hold_out_price/hold_in_price-1)

    # 7. 考虑换仓成本 
    if comm:
        chgpos_comm_perc = (1-comm)**abs(position - position.shift(1).fillna(0)) 
        hold_r = chgpos_comm_perc * (1+hold_r) - 1 
    # 8. 考虑杠杆 
    if leverage:
        hold_r *= leverage

    hold_win = hold_r>0
    hold_cumu_r = (1+hold_r).cumprod() - 1
    drawdown = (hold_cumu_r.cummax()-hold_cumu_r)/(1+hold_cumu_r).cummax()      
    ex_hold_r= hold_r-r0/(250*1440/data_period)
    data['hold_in_price'] = hold_in_price *abs(position)
    data['hold_out_price'] = hold_out_price *abs(position)
    data['hold_r'] = hold_r
    data['hold_win'] = hold_win
    data['hold_cumu_r'] = hold_cumu_r
    data['drawdown'] = drawdown
    data['ex_hold_r'] = ex_hold_r
    
    """       数值型特征 
        v_hold_cumu_r：         累计持仓收益
        v_pos_hold_times：      多仓开仓次数
        v_pos_hold_win_times：  多仓开仓盈利次数
        v_pos_hold_period：     多仓持有周期数
        v_pos_hold_win_period： 多仓持有盈利周期数
        v_neg_hold_times：      空仓开仓次数
        v_neg_hold_win_times：  空仓开仓盈利次数
        v_neg_hold_period：     空仓持有盈利周期数
        v_neg_hold_win_period： 空仓开仓次数
        
        v_hold_period：         持仓周期数（最后一笔未平仓订单也算）
        v_hold_win_period：     持仓盈利周期数（最后一笔未平仓订单也算）
        
        v_max_dd：              最大回撤
        v_annual_ret：          年化收益
        v_sharpe：              年化夏普
    """
    v_hold_cumu_r = hold_cumu_r.tolist()[-1]

    v_pos_hold_times= 0 
    v_pos_hold_win_times = 0
    v_pos_hold_period = 0
    v_pos_hold_win_period = 0
    v_neg_hold_times= 0 
    v_neg_hold_win_times = 0
    v_neg_hold_period = 0
    v_neg_hold_win_period = 0
    for w, r, pre_pos, pos in zip(hold_win, hold_r, position.shift(1), position):
        """ 当日有换仓（先结算上一次持仓，再初始化本次持仓） """
        if pre_pos!=pos: 
            # 判断pre_pos非空：若为空则是循环的第一次，此时无需结算，直接初始化持仓即可
            if pre_pos == pre_pos:
                # 结算上一次持仓
                if pre_pos>0:
                    v_pos_hold_times += 1
                    v_pos_hold_period += tmp_hold_period
                    v_pos_hold_win_period += tmp_hold_win_period
                    if tmp_hold_r>0:
                        v_pos_hold_win_times+=1
                elif pre_pos<0:
                    v_neg_hold_times += 1      
                    v_neg_hold_period += tmp_hold_period
                    v_neg_hold_win_period += tmp_hold_win_period
                    if tmp_hold_r>0:                    
                        v_neg_hold_win_times+=1
            # 初始化本次持仓
            tmp_hold_r = r
            tmp_hold_period = 0
            tmp_hold_win_period = 0
        else: 
            if abs(pos)>0:
                tmp_hold_period += 1
                if r>0:
                    tmp_hold_win_period += 1
                if abs(r)>0:
                    tmp_hold_r = (1+tmp_hold_r)*(1+r)-1       

    v_hold_period = (abs(position)>0).sum()
    v_hold_win_period = (hold_r>0).sum()
    v_max_dd = drawdown.max()    
    v_annual_std = ex_hold_r.std() * np.sqrt( len(data)/y_period ) 
    v_annual_ret = (1+v_hold_cumu_r) ** (1/y_period) - 1
    v_sharpe= v_annual_ret / v_annual_std

    """ 生成Performance DataFrame """
    performance_cols = ['累计收益', 
                        '多仓次数', '多仓成功次数', '多仓胜率', '多仓平均持有期', 
                        '空仓次数', '空仓成功次数',  '空仓胜率', '空仓平均持有期', 
                        '周期胜率', '最大回撤', '年化收益/最大回撤',
                        '年化收益','年化标准差', '年化夏普'
                       ]
    performance_values = ['{:.2%}'.format(v_hold_cumu_r),
                          v_pos_hold_times, v_pos_hold_win_times,
                                            '{:.2%}'.format(v_pos_hold_win_times/v_pos_hold_times if v_pos_hold_times != 0 else 0), 
                                            '{:.2f}'.format(v_pos_hold_period/v_pos_hold_times if v_pos_hold_times != 0 else 0),
                          v_neg_hold_times, v_neg_hold_win_times,
                                            '{:.2%}'.format(v_neg_hold_win_times / v_neg_hold_times if v_neg_hold_times != 0 else 0), 
                                            '{:.2f}'.format(v_neg_hold_period / v_neg_hold_times if v_neg_hold_times != 0 else 0),
                          '{:.2%}'.format(v_hold_win_period/v_hold_period if v_neg_hold_period != 0 else 0), 
                          '{:.2%}'.format(v_max_dd), 
                          '{:.2f}'.format(v_annual_ret/v_max_dd),
                          '{:.2%}'.format(v_annual_ret), 
                          '{:.2%}'.format(v_annual_std),
                          '{:.2f}'.format(v_sharpe)
                         ]
    performance_df = pd.DataFrame(performance_values, index=performance_cols)
    
    return data, performance_df

In [62]:
df_daily_1 = VHL(df_daily, sL=2)
df_signal = signal(df_daily_1,cL=15)
my_position_revenue = position(df_signal)
res_my_data_revenue, performance_df= statistic_performance(my_position_revenue, 
                                                   data_period=1, 
                                                   is_consider_open=True,
                                                   is_consider_stop=True,
                                                   is_daily_close=True,
                                                   point=0.4,
                                                   comm=0.0001)
print(performance_df)

                0
累计收益       31.87%
多仓次数          403
多仓成功次数        101
多仓胜率       25.06%
多仓平均持有期     10.28
空仓次数          541
空仓成功次数        112
空仓胜率       20.70%
空仓平均持有期      9.50
周期胜率       46.48%
最大回撤       11.45%
年化收益/最大回撤    1.60
年化收益       18.26%
年化标准差      14.68%
年化夏普         1.24


In [31]:
import datetime
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row, gridplot, layout
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

In [32]:
def visualize_performance(data):
    if 'trade_date' in data:
        data['trade_datetime'] = data['trade_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y%m%d'))
        dt = data['trade_datetime']
    else:
        dt = [datetime.datetime.strptime('{} {}'.format(d, t), '%Y-%m-%d %H:%M:%S') 
              for d, t in zip(data['date'], data['time'])]

    f1 = figure(height=300, width=700, sizing_mode='stretch_width', 
                title='Target Trend',
                x_axis_type='datetime',
                x_axis_label="trade_datetime", y_axis_label="close")
    f2 = figure(height=200, sizing_mode='stretch_width', 
                title='Position',
                x_axis_label="trade_datetime", y_axis_label="position",
                x_axis_type='datetime',
                x_range=f1.x_range)
    f3 = figure(height=200, sizing_mode='stretch_width', 
                title='Return',
                x_axis_type='datetime',
                x_range=f1.x_range)
    f4 = figure(height=200, sizing_mode='stretch_width', 
                title='Drawdown',
                x_axis_type='datetime',
                x_range=f1.x_range)


    # 绘制行情
    close = data['close']
    cumu_hold_close = (data['hold_cumu_r']+1) * close.tolist()[0]
    f1.line(dt, close, line_width=1)
    f1.line(dt, cumu_hold_close, line_width=1, color='red')

    # 绘制指标
    indi = figure(height=200, sizing_mode='stretch_width', 
                  title='KDJ',
                  x_axis_type='datetime',
                  x_range=f1.x_range
                 )


    # 绘制仓位
    position = data['position']
    f2.step(dt, position)

    # 绘制收益
    hold_r = data['hold_r']
    hold_cumu_r = data['hold_cumu_r']+1
    f3.line(dt, hold_cumu_r, line_width=1, color='red')
#     f3.vbar(x=dt, top=hold_r)

    # 绘制回撤
    drawdown = data['drawdown']
    f4.line(dt, -drawdown, line_width=1)

    #p = column(f1,f2,f3,f4)
    p = gridplot([ [f1],
                   #[indi],
                   [f2], 
                   [f3],
                   [f4]
                 ])
    show(p)

In [33]:
visualize_performance(res_my_data_revenue)

## 参数选取

In [64]:
#读取数据
df_daily = pd.read_csv(
    'IF8888.CCFX_5m.csv', sep=',', parse_dates=True)
# df_daily = df_daily.iloc[:, 0:3]
df_daily = df_daily[df_daily['date']<='2011-12-09']
performance = []
SL = np.arange(2,4)#2 15  2 4 6 21
CL = np.arange(6,21)
df_par = pd.DataFrame()
df_par["SL"]=np.zeros(30)
df_par["CL"]=np.zeros(30)
m1,m2 =[],[]
for n1 in SL:
    for n2 in CL:
#         print(n1,n2)
        df_daily_1 = VHL(df_daily, sL=n1)
        df_signal = signal(df_daily_1,cL=n2)
        my_position_revenue = position(df_signal)
        res_my_data_revenue, performance_df= statistic_performance(my_position_revenue, 
                                                           data_period=1, 
                                                           is_consider_open=True,
                                                           is_consider_stop=True,
                                                           is_daily_close=True,
                                                           point=0.4,
                                                           comm=0.0001)
#         print(float(performance_df.values[0][0].strip('%')))
        m1.append(int(n1))
        m2.append(int(n2))
        performance.append(float(performance_df.values[0][0].strip('%')))
        
df_par["SL"]=m1
df_par["CL"]=m2
df_par["performance"]=performance
df_par


,SL,CL,performance
0,2,6,31.80
1,2,7,31.80
2,2,8,31.80
3,2,9,31.80
4,2,10,31.80
5,2,11,31.80
6,2,12,31.80
7,2,13,31.80
8,2,14,31.80
9,2,15,31.87
